# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

Answer for question 1:
- The sum of the outputs of the generator for limit = 5 is 8.382332347441762
Answer for question 2:
- The 13th number yielded is 3.605551275463989

In [17]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

res = 0
for sqrt_value in generator:
    print(sqrt_value)
    res += sqrt_value
print(res)
print('---')
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
8.382332347441762
---
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [18]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [19]:
import dlt
import duckdb

# define the connection to load to.
# use duckdb
pipeline = dlt.pipeline(pipeline_name='workshop-homework', destination='duckdb', dataset_name='workshop-homework-dataset')

# load the first generator
info = pipeline.run(people_1(), table_name='people', write_disposition='replace')

# calculate the sum of ages of all people
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
res = conn.sql("SELECT SUM(Age) FROM workshop_homework_dataset.people").df()
display(res)

# append the second generator
info = pipeline.run(people_2(), table_name='people', write_disposition='append')
# calculate the sum of all ages of people
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
all_res = conn.sql("SELECT * FROM workshop_homework_dataset.people").df()
res = conn.sql("SELECT SUM(Age) FROM workshop_homework_dataset.people").df()
display(res)
display(all_res)

,sum(Age)
0,140.0


,sum(Age)
0,353.0


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707719825.615318,hAoJ+P4Rw4JGKw,None
1,2,Person_2,27,City_A,1707719825.615318,yVhoBdTcyV6YRA,None
2,3,Person_3,28,City_A,1707719825.615318,SYc29uPGvA8F+g,None
3,4,Person_4,29,City_A,1707719825.615318,Yh0GKRdNsLn4iw,None
4,5,Person_5,30,City_A,1707719825.615318,dN9GpOu7Tf3djA,None
5,3,Person_3,33,City_B,1707719826.1786602,YYr1zd7nK3+a0w,Job_3
6,4,Person_4,34,City_B,1707719826.1786602,AIw+VkXTJgGEyQ,Job_4
7,5,Person_5,35,City_B,1707719826.1786602,R8hGLSThiQ3EWw,Job_5
8,6,Person_6,36,City_B,1707719826.1786602,Ah9M8JK7+eVgLA,Job_6
9,7,Person_7,37,City_B,1707719826.1786602,aPbAnsVw1teIPw,Job_7


Load the first generator and calculate the sum of ages of all people.
Answer: The sum of ages of all people is 140
After correctly appending the data, calculate the sum of all ages of people.
Answer: The sum of all ages of people is 353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [16]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

UsageError: Line magic function `%%capture` not found.


In [20]:
# to do: homework :)

import dlt
import duckdb

# load the first generator
info = pipeline.run(people_1(), table_name='people', write_disposition='replace', primary_key='ID')
# append the second generator
info = pipeline.run(people_2(), table_name='people', write_disposition='merge', primary_key='ID')
# calculate the sum of all ages of people
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
all_res = conn.sql("SELECT * FROM workshop_homework_dataset.people").df()
res = conn.sql("SELECT SUM(Age) FROM workshop_homework_dataset.people").df()
display(res)
display(all_res)

,sum(Age)
0,266.0


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707719831.326818,9hejOrG7JD8LIw,None
1,2,Person_2,27,City_A,1707719831.326818,NlJrJUR7mAPlcg,None
2,7,Person_7,37,City_B,1707719831.909038,nEeDKwdnyFKiZg,Job_7
3,4,Person_4,34,City_B,1707719831.909038,Z9K76Qme2JFFtA,Job_4
4,5,Person_5,35,City_B,1707719831.909038,LFl5iY/3f13QlQ,Job_5
5,3,Person_3,33,City_B,1707719831.909038,YUnTxdRgwhE50w,Job_3
6,6,Person_6,36,City_B,1707719831.909038,7m/bGWAWJTOitA,Job_6
7,8,Person_8,38,City_B,1707719831.909038,Q1Utm0gV5HhgYA,Job_8


In [ ]:
# The sum of ages of all the people loaded as described above is 266

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX